# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-08-16&until-occurred-date=2021-08-23


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

26130 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-16&until-occurred-date=2021-08-23
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=f19c203a-cd32-4b9d-b461-de9708549e4e&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-16&until-occurred-date=2021-08-23
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=1fd9b4c5-bc42-4db1-81fd-46de0a1a481d&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-08-16&until-occurred-date=2021-08-23
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-07-26.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.13.452194...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.04.19.21255739...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.08.19.456982...
REST API query complete  200
REST API query started for 10.1101/2021.08.13.21261989...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2020.08.10.20172288...
REST API query complete  200
REST API query started for 10.1101/2021.08.12.456173...
REST API query complete  200
REST API query started for 10.1101/2021.08.12.21261991...
REST API query complete  200
REST API query started for 10.1101/2021.05.18.21257385...
REST API query complete  200
REST API query started for 10.1101/2021.08.11.212

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.08.19.456982,812,bioRxiv,Genomics,False,Somatic mutation rates scale with lifespan acr...,"[{'name': 'Alex Cagan'}, {'name': 'Adrian Baez...",<p>The rates and patterns of somatic mutation ...,2021-08-19
1,10.1101/2021.08.16.21261912,190,medRxiv,Infectious Diseases (except HIV/AIDS),True,Genomic profiles of Covishield and Covaxin bre...,"[{'name': 'Arup Ghosh'}, {'name': 'Safal Walia...",<p>Vaccine breakthrough infections pose a vast...,2021-08-20
2,10.1101/2021.08.15.21262067,102,medRxiv,Infectious Diseases (except HIV/AIDS),True,Significant reduction in humoral immunity amon...,"[{'name': 'David H. Canaday'}, {'name': 'Olada...",<p>High COVID-19 mortality among nursing home ...,2021-08-18
3,10.1101/2021.08.16.456551,59,bioRxiv,Molecular Biology,False,Tuning levels of low-complexity domain interac...,"[{'name': 'Shasha Chong'}, {'name': 'Thomas G....",<p>Gene activation by mammalian transcription ...,2021-08-17
4,10.1101/2021.08.20.457082,44,bioRxiv,Microbiology,False,A nitrite-oxidizing bacterium constitutively c...,"[{'name': 'Pok Man Leung'}, {'name': 'Anne Dae...",<p>Chemolithoautotrophic nitrite-oxidizing bac...,2021-08-20
5,10.1101/2021.08.20.457057,42,bioRxiv,Bioinformatics,False,MultiVI: deep generative model for the integra...,"[{'name': 'Tal Ashuach'}, {'name': 'Mariano I ...",<p>The ability to jointly profile the transcri...,2021-08-20
6,10.1101/2021.08.17.21262195,41,medRxiv,Epidemiology,True,COVID-19 Incidence and Hospitalization Rates a...,[{'name': 'Jeffrey E Harris'}],<p>We tested whether COVID-19 incidence and ho...,2021-08-20
7,10.1101/2021.08.20.457079,40,bioRxiv,Neuroscience,False,Humans and mice fluctuate between external and...,"[{'name': 'Veith Andreas Weilnhammer'}, {'name...",<p>Perception cycles through periods of enhanc...,2021-08-20
8,10.1101/2021.08.21.457186,40,bioRxiv,Evolutionary Biology,False,Evidence of long-term purging of mutation load...,"[{'name': 'Andrew D Foote'}, {'name': 'Thomas ...",<p>Genomes of high latitude killer whales harb...,2021-08-21
9,10.1101/2021.08.16.456499,36,bioRxiv,Bioinformatics,False,scFlow: A Scalable and Reproducible Analysis P...,"[{'name': 'Combiz Khozoie'}, {'name': 'Nurun F...",<p>Advances in single-cell RNA-sequencing tech...,2021-08-19


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

60 preprints (including 10 covering SARS-CoV-2, 52 from bioRxiv and 8 from medRxiv) published in the last 7 days before 2021-08-23 had been tweeted at least 3 times (maximum 812).